In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.callbacks import Callback
import pandas as pd
import random
# train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')

# Select 100 random samples
# train_ratings = train_ratings.sample(n=10, random_state=42)
ratings = torch.tensor([float(random.randint(1, 5)) for _ in range(5)])


In [8]:
titles = ["Inception", "Titanic", "Avatar", "The Matrix", "The Godfather", "Star Wars", "The Dark Knight", "Pulp Fiction", "Forrest Gump", "Jurassic Park"]
genres = ["Action", "Drama", "Sci-Fi", "Romance", "Thriller", "Comedy", "Adventure", "Fantasy", "Horror", "Mystery"]
years = ["1994", "1999", "2008", "2010", "2014", "2018", "2020", "1984", "1995", "2001"]

movies = []
for _ in range(5):
    movie = random.choice(titles)
    genre = random.choice(genres)
    year = random.choice(years)
    sample = f"title: {movie} [SEP] genre: {genre} [SEP] year: {year}"
    movies.append(sample)

In [ ]:
import random

occupations = ["doctor", "engineer", "teacher", "nurse", "writer", "musician", "artist", "lawyer", "scientist", "pilot"]
ages = ["18-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75+"]
genders = ["Male", "Female"]

# Generate 10 random samples
users = []
for _ in range(5):
    occupation = random.choice(occupations)
    age = random.choice(ages)
    gender = random.choice(genders)
    sample = f"profession: {occupation} [SEP] age range: {age} [SEP] gender: {gender}"
    users.append(sample)

In [178]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [9]:
users = [
    "profession: teacher [SEP] age range: 65-74",
    "profession: musician [SEP] age range: 75+",
    "profession: teacher [SEP] age range: 25-55",
    "profession: musician [SEP] age range: 65-74",
    "profession: teacher [SEP] age range: 45-55",
]

In [34]:
users = pd.read_csv('processed_dataset/MovieLens-1M/users/users_movielens.csv')
movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens.csv')
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/valdata_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/testdata_movielens.csv')

In [35]:
# Combine user features into a single string for each user
users['user_features'] = 'occupation: ' + users['occupation'] + ' [SEP] age: ' + users['age'].astype(str) + ' [SEP] gender: ' + users['gender'].astype(str)

# Combine movie features into a single string for each movie
movies['movie_features'] = 'title: ' + movies['title'] + ' [SEP] genres: ' + movies['genres']

In [36]:
# Create a dictionary for fast lookup
user_features_dict = users.set_index('user_id')['user_features'].to_dict()
movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()

# Create lists of user and item texts
train_user_texts = [user_features_dict[userId] for userId in train_ratings['user_id']]
train_item_texts = [movie_features_dict[movieId] for movieId in train_ratings['item_id'].unique()]

# Create lists of user and item texts
val_user_texts = [user_features_dict[userId] for userId in val_ratings['user_id']]
val_item_texts = [movie_features_dict[movieId] for movieId in val_ratings['item_id'].unique()]

train_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(train_ratings['item_id'].unique())}
val_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(val_ratings['item_id'].unique())}
test_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(test_ratings['item_id'].unique())}

test_item_texts = [movie_features_dict[movieId] for movieId in test_ratings['item_id'].unique()]

In [7]:
train_item_texts[0]

'title: Silence of the Lambs, The (1991) [SEP] genres: Drama|Thriller'

In [408]:
movie_features_dict[1434]

'title: Stranger, The (1994) [SEP] genres: Action'

In [396]:
len(movie_features_dict)

3883

In [399]:
len(item_texts)

3706

In [304]:
len(val_user_texts)

200042

In [37]:
val_ratings

,user_id,item_id,rating,timestamp
0,6040,858,4,956703932
1,6040,3111,5,956704056
2,6040,213,5,956704056
3,6040,608,4,956704475
4,6040,1649,5,956704519
...,...,...,...,...
200037,4958,2453,4,1046454260
200038,4958,2043,1,1046454282
200039,4958,2399,1,1046454338
200040,4958,2634,3,1046454548


In [39]:
val_user_texts[0]

'occupation: doctor/health care [SEP] age: 25-34 [SEP] gender: Male'

In [45]:
# train_item_texts[item_id_to_idx.get(train_ratings['item_id'][0])]
item_id = 858
text = val_item_texts[val_item_id_to_idx[item_id]] if item_id in val_item_id_to_idx else "Item ID not found"
text

'title: Godfather, The (1972) [SEP] genres: Action|Crime|Drama'

In [44]:
val_item_id_to_idx

{858: 0,
 3111: 1,
 213: 2,
 608: 3,
 1649: 4,
 246: 5,
 866: 6,
 448: 7,
 2324: 8,
 58: 9,
 1834: 10,
 2391: 11,
 2067: 12,
 3022: 13,
 1269: 14,
 2804: 15,
 2791: 16,
 1197: 17,
 935: 18,
 3363: 19,
 1265: 20,
 3396: 21,
 3471: 22,
 924: 23,
 1097: 24,
 919: 25,
 2565: 26,
 1282: 27,
 899: 28,
 1947: 29,
 364: 30,
 588: 31,
 2946: 32,
 1380: 33,
 1083: 34,
 1212: 35,
 923: 36,
 920: 37,
 232: 38,
 1223: 39,
 1221: 40,
 1250: 41,
 908: 42,
 1299: 43,
 1207: 44,
 3469: 45,
 1721: 46,
 3468: 47,
 1945: 48,
 3359: 49,
 1231: 50,
 1949: 51,
 2683: 52,
 2707: 53,
 2826: 54,
 1203: 55,
 3504: 56,
 1276: 57,
 2555: 58,
 3285: 59,
 2710: 60,
 2975: 61,
 2289: 62,
 2038: 63,
 3016: 64,
 1293: 65,
 2722: 66,
 2546: 67,
 1911: 68,
 2575: 69,
 926: 70,
 2303: 71,
 785: 72,
 1953: 73,
 235: 74,
 968: 75,
 2723: 76,
 3325: 77,
 2502: 78,
 1956: 79,
 2560: 80,
 2396: 81,
 2762: 82,
 2840: 83,
 2491: 84,
 2702: 85,
 2693: 86,
 1623: 87,
 50: 88,
 1198: 89,
 3149: 90,
 1148: 91,
 1267: 92,
 3006: 93,


In [381]:
print(item_id_to_idx.get(1433))

3070


In [383]:
train_item_texts[3070]

'title: Machine, The (1994) [SEP] genres: Comedy|Horror'

In [46]:
val_ratings['rating'][0]

4

In [240]:
values_list = list(item_id_to_idx.values())
values_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [238]:
item_texts[0]

'title: Silence of the Lambs, The (1991) [SEP] genres: Drama|Thriller'

In [218]:
len(user_texts)


600125

In [212]:
# # Create a dictionary for fast lookup
# user_features_dict = users.set_index('user_id')['user_features'].to_dict()
# movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()
#
# # Create lists of user and item texts
# user_texts = [user_features_dict[userId] for userId in train_ratings['user_id']]
# item_texts = [movie_features_dict[movieId] for movieId in train_ratings['item_id'].unique()]
#
# # Create a mapping from userId and movieId to indices
# user_id_to_idx = {userId: idx for idx, userId in enumerate(train_ratings['user_id'])}
# movie_id_to_idx = {movieId: idx for idx, movieId in enumerate(train_ratings['item_id'].unique())}
#
# # Map userId and movieId in ratings_df to indices
# train_ratings['user_idx'] = train_ratings['user_id'].map(user_id_to_idx)
# train_ratings['movie_idx'] = train_ratings['item_id'].map(movie_id_to_idx)
#
# # Map userId and movieId in ratings_val to indices
# val_ratings['user_idx'] = val_ratings['user_id'].map(user_id_to_idx)
# val_ratings['movie_idx'] = val_ratings['item_id'].map(movie_id_to_idx)
#
# # Extract user indices, item indices, and ratings
# train_user_indices = torch.LongTensor(train_ratings['user_idx'].values).to(device)
# train_item_indices = torch.LongTensor(train_ratings['movie_idx'].values).to(device)
# train_labels = torch.FloatTensor(train_ratings['rating'].values).to(device)
#
# # Extract user indices, item indices, and ratings for validation
# val_user_indices = torch.LongTensor(val_ratings['user_idx'].values).to(device)
# val_item_indices = torch.LongTensor(val_ratings['movie_idx'].values).to(device)
# val_labels = torch.FloatTensor(val_ratings['rating'].values).to(device)

In [180]:
users['user_features']

0       occupation: K-12 student [SEP] age: Under 18 [...
1       occupation: self-employed [SEP] age: 56+ [SEP]...
2       occupation: scientist [SEP] age: 25-34 [SEP] g...
3       occupation: executive/managerial [SEP] age: 45...
4       occupation: writer [SEP] age: 25-34 [SEP] gend...
                              ...                        
6035    occupation: scientist [SEP] age: 25-34 [SEP] g...
6036    occupation: academic/educator [SEP] age: 45-49...
6037    occupation: academic/educator [SEP] age: 56+ [...
6038    occupation: other or not specified [SEP] age: ...
6039    occupation: doctor/health care [SEP] age: 25-3...
Name: user_features, Length: 6040, dtype: object

In [163]:
ratings = torch.tensor([2., 4., 1., 5., 1.])

In [164]:
ratings

tensor([2., 4., 1., 5., 1.])

In [10]:

class UserItemDataset(Dataset):
    def __init__(self, users, items, ratings):
        self.users = users
        self.items = items
        self.ratings = ratings
    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return {
            'user': self.users[idx],
            'item': self.items[idx],
            'rating': self.ratings[idx]
        }

In [72]:
data_length = len(train_user_texts)
random_indices = random.sample(range(data_length), 10)

random_user_texts = [train_user_texts[i] for i in random_indices]
random_item_ids = [train_ratings['item_id'][i] for i in random_indices]
random_ratings = [train_ratings['rating'][i] for i in random_indices]

In [76]:
random_indices = random.sample(range(data_length), 10)

val_random_user_texts = [train_user_texts[i] for i in random_indices]
val_random_item_ids = [train_ratings['item_id'][i] for i in random_indices]
val_random_ratings = [train_ratings['rating'][i] for i in random_indices]

In [80]:
random_item_ids

[3052, 233, 3208, 52, 2858, 2985, 3755, 653, 921, 2291]

In [81]:
# dataset = UserItemDataset(users, movies, ratings)
train_dataset = UserItemDataset(random_user_texts, random_item_ids, random_ratings)
val_dataset = UserItemDataset(val_random_user_texts, val_random_item_ids, val_random_ratings)

# train_dataset = UserItemDataset(train_user_texts[:10], train_ratings['item_id'], train_ratings['rating'][:10])
# val_dataset = UserItemDataset(val_user_texts[:5], val_ratings['item_id'], val_ratings['rating'][:5])

# dataset[0]

In [48]:
train_dataset.users[0]

'occupation: doctor/health care [SEP] age: 25-34 [SEP] gender: Male'

In [49]:
train_dataset.items[0]

593

In [50]:
train_dataset.ratings[0]

5

In [86]:

class TwoTowerModel(pl.LightningModule):
    def __init__(self, user_model_name, item_model_name, embedding_size=384):
        super(TwoTowerModel, self).__init__()
        self.user_model = SentenceTransformer(user_model_name)
        self.item_model = SentenceTransformer(item_model_name)

        self.user_fc = nn.Linear(embedding_size, embedding_size)
        self.item_fc = nn.Linear(embedding_size, embedding_size)

        self.criterion = nn.MSELoss()
        self.epoch_losses = {'train_loss': [], 'val_loss': []}

    def forward(self, user_text, item_text, step):
        if step == 'train':
            item_text = [train_item_texts[train_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]
        elif step == 'val':
            item_text = [val_item_texts[val_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]
        else:
            item_text = [test_item_texts[test_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]

        # print(user_text, item_text)
        user_embedding = self.user_model.encode(user_text, convert_to_tensor=True)
        item_embedding = self.item_model.encode(item_text, convert_to_tensor=True)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embedding)
        # Log embeddings
        # self.user_emb = user_embedding
        # self.item_emb = item_embedding
        # print("User Embedding:", user_embedding)
        # print("Item Embedding:", item_embedding)
        # print("User Output:", user_output.size())
        # print("Item Output:", item_output.size())
        # Compute the dot product for each user-item pair in the batch
        dot_product = torch.sum(user_output * item_output, dim=1)
        # print("Dot Product:", dot_product)

        return dot_product

    def training_step(self, batch, batch_idx):
        user_texts = batch['user']
        item_texts = batch['item']
        ratings = batch['rating'].float()

        preds = self(user_texts, item_texts, 'train')
        preds = 4 * torch.sigmoid(preds) + 1
        # print(user_texts, item_texts, preds, ratings)
        loss = self.criterion(preds, ratings)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        user_texts = batch['user']
        item_texts = batch['item']
        ratings = batch['rating'].float()
        print(item_texts)
        preds = self(user_texts, item_texts, 'val')
        preds = 4 * torch.sigmoid(preds) + 1

        loss = self.criterion(preds, ratings)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-5)

class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            pl_module.epoch_losses['train_loss'].append(train_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss: {train_loss.item()}")

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            pl_module.epoch_losses['val_loss'].append(val_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Val Loss: {val_loss.item()}")

# Example usage
# users = ["I love action movies", "Comedy movies are my favorite"]
# items = ["An action-packed thriller", "A hilarious comedy"]
# ratings = torch.tensor([5.0, 4.0])

train_loader = DataLoader(train_dataset, batch_size=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=2, drop_last=True)  # For demonstration purposes, using the same data for validation

model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2')

trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1, callbacks=[PrintLossesCallback()], enable_progress_bar=False)
trainer.fit(model, train_loader, val_loader)

# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    train_loss = model.epoch_losses['train_loss'][epoch] if epoch < len(model.epoch_losses['train_loss']) else 'N/A'
    val_loss = model.epoch_losses['val_loss'][epoch] if epoch < len(model.epoch_losses['val_loss']) else 'N/A'
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")


D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | user_model | SentenceTransformer | 22.7 M | train
1 | item_model | SentenceTransformer | 22.7 M | train
2 | user_fc    | Linear              | 147 K  | train
3 | item_fc    | Linear              | 147 K  | train
4 | criterion  | MSELoss             | 0      | train
-----------------------------------------------------------
45.7 M    Trainable params
0         Non-trainable params
45.7 M    Total params
182.888   Total estimated model params siz

tensor([2378,  764], device='cuda:0')
tensor([2294,  785], device='cuda:0')
Epoch 1: Val Loss: 2.638620376586914
tensor([2378,  764], device='cuda:0')
tensor([2294,  785], device='cuda:0')
tensor([2321, 1394], device='cuda:0')
tensor([3752, 1269], device='cuda:0')
tensor([1210, 2302], device='cuda:0')
Epoch 1: Val Loss: 2.444434642791748
Epoch 1: Train Loss: 0.2743901014328003
tensor([2378,  764], device='cuda:0')
tensor([2294,  785], device='cuda:0')
tensor([2321, 1394], device='cuda:0')
tensor([3752, 1269], device='cuda:0')
tensor([1210, 2302], device='cuda:0')
Epoch 2: Val Loss: 2.447988986968994
Epoch 2: Train Loss: 0.22457553446292877
tensor([2378,  764], device='cuda:0')
tensor([2294,  785], device='cuda:0')
tensor([2321, 1394], device='cuda:0')
tensor([3752, 1269], device='cuda:0')
tensor([1210, 2302], device='cuda:0')
Epoch 3: Val Loss: 2.405057430267334
Epoch 3: Train Loss: 0.16349312663078308
tensor([2378,  764], device='cuda:0')
tensor([2294,  785], device='cuda:0')
tensor([

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch losses:
Epoch 1: Train Loss: 0.2743901014328003, Val Loss: 2.638620376586914
Epoch 2: Train Loss: 0.22457553446292877, Val Loss: 2.444434642791748
Epoch 3: Train Loss: 0.16349312663078308, Val Loss: 2.447988986968994
Epoch 4: Train Loss: 0.11080898344516754, Val Loss: 2.405057430267334
Epoch 5: Train Loss: 0.0744146928191185, Val Loss: 2.3464157581329346


In [45]:
model.epoch_losses

{'train_loss': [1.1349900960922241,
  1.1209874153137207,
  1.1229170560836792,
  1.1273095607757568,
  1.1311087608337402,
  1.1337617635726929,
  1.1353812217712402,
  1.1362100839614868,
  1.1364727020263672,
  1.1363470554351807],
 'val_loss': [1.5670127868652344,
  1.1985464096069336,
  1.1731033325195312,
  1.16090726852417,
  1.1534429788589478,
  1.1482261419296265,
  1.144281268119812,
  1.1411384344100952,
  1.1385533809661865,
  1.13637375831604,
  1.1345018148422241]}

In [87]:
def get_top_n_recommendations(user_text, model, item_dataset, n=5):
    item_texts = [item_dataset[i] for i in range(len(item_dataset))]
    item_texts= torch.tensor(item_texts)
    # print(item_texts)
    preds = model.forward(user_text, item_texts, 'test')
    preds = 4 * torch.sigmoid(preds) + 1
    top_n_indices = torch.argsort(preds, descending=True)[:n]
    top_n_items = [item_dataset[i] for i in top_n_indices]
    top_n_scores = preds[top_n_indices]
    return list(zip(top_n_items, top_n_scores))

In [112]:
# Example user text
# user_text = "profession: doctor [SEP] age range: 65-74 [SEP] gender: Male"
user_text = "profession: scientist [SEP] age range: 75+ [SEP] gender: Male"

top_recommendations = get_top_n_recommendations(user_text, model, random_item_ids, n=10)
# top_recommendations = get_top_n_recommendations(user_text, model, movies, n=5)


tensor([3052,  233, 3208,   52, 2858, 2985, 3755,  653,  921, 2291])


In [97]:
t_item_texts = [movie_features_dict[movieId] for movieId in random_item_ids]
t_item_texts

['title: Dogma (1999) [SEP] genres: Comedy',
 'title: Exotica (1994) [SEP] genres: Drama',
 'title: Loaded Weapon 1 (1993) [SEP] genres: Action|Comedy',
 'title: Mighty Aphrodite (1995) [SEP] genres: Comedy',
 'title: American Beauty (1999) [SEP] genres: Comedy|Drama',
 'title: Robocop (1987) [SEP] genres: Action|Crime|Sci-Fi',
 'title: Perfect Storm, The (2000) [SEP] genres: Action|Adventure|Thriller',
 'title: Dragonheart (1996) [SEP] genres: Action|Adventure|Fantasy',
 'title: My Favorite Year (1982) [SEP] genres: Comedy',
 'title: Edward Scissorhands (1990) [SEP] genres: Drama|Romance']

In [106]:
[f"{item_id} {text}" for item_id, text in zip(random_item_ids, t_item_texts)]

['3052 title: Dogma (1999) [SEP] genres: Comedy',
 '233 title: Exotica (1994) [SEP] genres: Drama',
 '3208 title: Loaded Weapon 1 (1993) [SEP] genres: Action|Comedy',
 '52 title: Mighty Aphrodite (1995) [SEP] genres: Comedy',
 '2858 title: American Beauty (1999) [SEP] genres: Comedy|Drama',
 '2985 title: Robocop (1987) [SEP] genres: Action|Crime|Sci-Fi',
 '3755 title: Perfect Storm, The (2000) [SEP] genres: Action|Adventure|Thriller',
 '653 title: Dragonheart (1996) [SEP] genres: Action|Adventure|Fantasy',
 '921 title: My Favorite Year (1982) [SEP] genres: Comedy',
 '2291 title: Edward Scissorhands (1990) [SEP] genres: Drama|Romance']

In [107]:
# data = {
#     "User": users,  # Repeating users to match the number of ratings
#     "Movie": movies,
#     "Rating": ratings
# }
data = {
    "User": random_user_texts,  # Repeating users to match the number of ratings
    "Movie": [f"{item_id} {text}" for item_id, text in zip(random_item_ids, t_item_texts)],
    "Rating": random_ratings
}


In [108]:
df = pd.DataFrame(data)


In [109]:
df

,User,Movie,Rating
0,occupation: unemployed [SEP] age: 18-24 [SEP] ...,3052 title: Dogma (1999) [SEP] genres: Comedy,3
1,occupation: writer [SEP] age: 45-49 [SEP] gend...,233 title: Exotica (1994) [SEP] genres: Drama,2
2,occupation: other or not specified [SEP] age: ...,3208 title: Loaded Weapon 1 (1993) [SEP] genre...,2
3,occupation: technician/engineer [SEP] age: 18-...,52 title: Mighty Aphrodite (1995) [SEP] genres...,1
4,occupation: scientist [SEP] age: 25-34 [SEP] g...,2858 title: American Beauty (1999) [SEP] genre...,3
5,occupation: technician/engineer [SEP] age: 25-...,2985 title: Robocop (1987) [SEP] genres: Actio...,3
6,occupation: college/grad student [SEP] age: 18...,"3755 title: Perfect Storm, The (2000) [SEP] ge...",1
7,occupation: other or not specified [SEP] age: ...,653 title: Dragonheart (1996) [SEP] genres: Ac...,4
8,occupation: scientist [SEP] age: 25-34 [SEP] g...,921 title: My Favorite Year (1982) [SEP] genre...,3
9,occupation: academic/educator [SEP] age: 25-34...,2291 title: Edward Scissorhands (1990) [SEP] g...,3


In [ ]:
user_text = "profession: scientist [SEP] age range: 75+ [SEP] gender: Male"

In [113]:
top_recommendations

[(921, tensor(2.9823, grad_fn=<UnbindBackward0>)),
 (653, tensor(2.7847, grad_fn=<UnbindBackward0>)),
 (3052, tensor(2.7314, grad_fn=<UnbindBackward0>)),
 (3208, tensor(2.3732, grad_fn=<UnbindBackward0>)),
 (2985, tensor(2.3191, grad_fn=<UnbindBackward0>)),
 (2858, tensor(2.2966, grad_fn=<UnbindBackward0>)),
 (3755, tensor(2.2466, grad_fn=<UnbindBackward0>)),
 (52, tensor(2.2301, grad_fn=<UnbindBackward0>)),
 (2291, tensor(2.1117, grad_fn=<UnbindBackward0>)),
 (233, tensor(1.7996, grad_fn=<UnbindBackward0>))]

In [104]:
model.user_emb

tensor([[ 8.3276e-01,  8.2051e-02,  3.0309e-01,  2.1005e-01, -3.0842e-01,
          8.1448e-01, -9.2465e-02, -3.7289e-01,  8.7239e-02,  6.1374e-01,
         -1.7548e-01, -2.6898e-01,  1.0095e-01,  2.0776e-02, -2.9030e-02,
          3.1094e-01, -1.5206e-01, -1.8223e-01,  7.2254e-01,  2.6763e-01,
         -1.7961e-01,  2.1395e-01,  2.5993e-01, -6.7496e-01,  5.0078e-01,
         -6.1833e-01, -2.8589e-01,  1.1161e-01, -2.1280e-01, -2.1083e-01,
          3.3463e-01,  4.1669e-01,  4.4386e-01, -1.3550e-02,  5.8307e-02,
         -1.3067e-01, -1.0299e+00,  1.5860e-01, -2.2157e-02,  3.0831e-01,
         -4.3915e-02, -7.7502e-02, -1.4880e-01, -6.4330e-01, -1.1070e-02,
          1.3839e-01, -1.5811e-01, -4.4167e-04,  1.1641e-01,  3.9993e-01,
         -9.0280e-01, -6.9036e-01,  1.4046e-01,  3.3495e-01, -3.1607e-01,
         -4.2657e-01,  7.9768e-03,  2.0949e-02,  2.9584e-01,  6.0345e-02,
         -2.0942e-01,  3.5031e-01, -7.5630e-01, -7.8596e-02,  1.1698e-02,
          1.8370e-01,  2.7463e-01, -2.

In [105]:
model.item_emb

tensor([[ 3.8231e-02, -5.1786e-01,  2.8932e-01, -6.5171e-02, -2.2142e-01,
          4.8638e-01, -3.7634e-01, -1.0395e-03,  5.1911e-02,  2.2404e-01,
          1.9158e-01, -1.5225e-01, -6.5987e-01,  3.5734e-02,  1.8200e-01,
         -1.5093e-01,  4.0238e-01,  6.6269e-02,  6.9296e-01,  5.6623e-01,
          1.0681e-01,  6.3442e-01,  2.1215e-01, -6.2358e-01,  1.9043e-01,
         -5.2030e-01,  1.6891e-01,  3.1781e-01, -6.8459e-01, -1.2312e-01,
          4.7328e-01,  4.6238e-02, -2.1466e-01,  5.2682e-02,  1.6736e-01,
          1.9360e-01,  3.4173e-01,  2.5968e-01,  2.0081e-01, -5.5312e-01,
         -3.8487e-01,  2.2561e-01,  1.7053e-01,  3.6520e-01, -2.2282e-01,
         -1.8844e-01, -3.3730e-01, -5.5030e-01,  5.9279e-01,  2.9978e-01,
         -6.6722e-01, -1.8076e-01, -3.6294e-01,  1.3673e-01,  6.3215e-01,
         -4.3464e-02,  1.1103e-01,  3.4903e-01,  9.4543e-02,  1.7410e-01,
          2.7457e-01, -1.4252e-01, -6.0077e-01,  1.5249e-01,  1.1750e-01,
         -4.0101e-01,  1.4326e-02,  4.

In [107]:
# Calculate the dot product
dot_product = torch.sum(model.user_emb * model.item_emb, dim=1)

print("Dot product:", dot_product.item())

Dot product: 16.750228881835938


In [108]:
# Calculate the dot product
dot_product = torch.sum(model.user_emb *  model.item_emb, dim=1)

# Calculate magnitudes
user_magnitude = torch.norm(model.user_emb, dim=1)
item_magnitude = torch.norm( model.item_emb, dim=1)

# Theoretical maximum and minimum dot product values
max_dot_product = user_magnitude * item_magnitude
min_dot_product = -max_dot_product

# Display results
print(f"Dot product: {dot_product}")
print(f"User embedding magnitude: {user_magnitude}")
print(f"Item embedding magnitude: {item_magnitude}")
print(f"Theoretical max dot product: {max_dot_product}")
print(f"Theoretical min dot product: {min_dot_product}")

Dot product: tensor([16.7502], device='cuda:0')
User embedding magnitude: tensor([7.4314], device='cuda:0')
Item embedding magnitude: tensor([7.2215], device='cuda:0')
Theoretical max dot product: tensor([53.6660], device='cuda:0')
Theoretical min dot product: tensor([-53.6660], device='cuda:0')
